In [34]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [89]:
data_dir = 'data/'

def read_text(file):
    with open(file, 'r') as f:
        text = f.readlines()
    for i in range(len(text)):
        text[i] = text[i].replace('\n', '')
    return text

palindromic_strings = read_text(data_dir + 'palindromic_strings.txt')
nonpalindromic_strings = read_text(data_dir + 'nonpalindromic_strings.txt')
palindromic_strings_off_by_one_swap = read_text(data_dir + 'palindromic_strings_off_by_one_swap.txt')
palindromic_strings_off_by_one_random_replacement = read_text(data_dir + 'palindromic_strings_off_by_one_random_replacement.txt') 

In [91]:
challenge_dataset = palindromic_strings[-3000:] + nonpalindromic_strings[-1000:] + palindromic_strings_off_by_one_swap[-1000:] + palindromic_strings_off_by_one_random_replacement[-1000:]
challenge_labels = [0] * 3000 + [1] * 1000 + [1] * 1000 + [1] * 1000
challenge_finegrained_labels = {}
for i in range(3000):
    challenge_finegrained_labels[palindromic_strings[-3000:][i]] = 0
for i in range(1000):
    challenge_finegrained_labels[nonpalindromic_strings[-1000:][i]] = 1
    challenge_finegrained_labels[palindromic_strings_off_by_one_swap[-1000:][i]] = 2
    challenge_finegrained_labels[palindromic_strings_off_by_one_random_replacement[-1000:][i]] = 3

dataset = palindromic_strings[:30000] + nonpalindromic_strings[:10000] + palindromic_strings_off_by_one_swap[:10000] + palindromic_strings_off_by_one_random_replacement[:10000]
labels = [0] * 30000 + [1] * 10000 + [1] * 10000 + [1] * 10000
finegrained_labels = {}
for i in range(30000):
    finegrained_labels[palindromic_strings[:30000][i]] = 0
for i in range(10000):
    finegrained_labels[nonpalindromic_strings[:10000][i]] = 1
    finegrained_labels[palindromic_strings_off_by_one_swap[:10000][i]] = 2
    finegrained_labels[palindromic_strings_off_by_one_random_replacement[:10000][i]] = 3
print(len(dataset), len(labels))
# train test split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42, stratify=labels)

40000 4000


ValueError: Found input variables with inconsistent numbers of samples: [40000, 4000]

In [ ]:
train_dataset_finegrained_labels = []
for i in train_dataset:
    train_dataset_finegrained_labels.append(finegrained_labels[i])
test_dataset_finegrained_labels = []
for i in test_dataset:
    test_dataset_finegrained_labels.append(finegrained_labels[i])

NameError: name 'finegrained_labels' is not defined

In [ ]:
SEQ_LENGTH = 9
all_chars = string.ascii_uppercase

# Encode Using Alphabet Arrays With Bools

In [52]:
def encode(seq):
    seq = seq.upper()
    assert len(seq) == SEQ_LENGTH, "length of sequence must be {}".format(SEQ_LENGTH)
    encoding = [0]*len(all_chars)
    for i, char in enumerate(seq):
        encoding[all_chars.index(char)] = 1
    return encoding

encoded_train_dataset = [encode(seq) for seq in train_dataset]
encoded_test_dataset = [encode(seq) for seq in test_dataset]
encoded_challenge_dataset = [encode(seq) for seq in challenge_dataset]

In [53]:
clf = MLPClassifier(hidden_layer_sizes=(10), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
clf.fit(encoded_train_dataset, train_labels)

Iteration 1, loss = 0.87485018
Validation score: 0.500000
Iteration 2, loss = 0.77278844
Validation score: 0.500000
Iteration 3, loss = 0.71823541
Validation score: 0.501250
Iteration 4, loss = 0.67151861
Validation score: 0.648750
Iteration 5, loss = 0.63646036
Validation score: 0.755000
Iteration 6, loss = 0.61691422
Validation score: 0.711250
Iteration 7, loss = 0.60566679
Validation score: 0.700000
Iteration 8, loss = 0.59577298
Validation score: 0.706250
Iteration 9, loss = 0.58535367
Validation score: 0.717500
Iteration 10, loss = 0.57381297
Validation score: 0.737500
Iteration 11, loss = 0.56105056
Validation score: 0.765000
Iteration 12, loss = 0.54675995
Validation score: 0.783750
Iteration 13, loss = 0.53130545
Validation score: 0.820000
Iteration 14, loss = 0.51364311
Validation score: 0.851250
Iteration 15, loss = 0.49418075
Validation score: 0.862500
Iteration 16, loss = 0.47307181
Validation score: 0.883750
Iteration 17, loss = 0.45029444
Validation score: 0.908750
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=10, max_iter=1000,
              random_state=42, verbose=True)

In [54]:
test_preds = clf.predict(encoded_test_dataset)
test_preds = classification_report(test_labels, test_preds, output_dict=True)

{'0': {'precision': 1.0,
  'recall': 0.987,
  'f1-score': 0.9934574735782586,
  'support': 1000.0},
 '1': {'precision': 0.9871668311944719,
  'recall': 1.0,
  'f1-score': 0.9935419771485345,
  'support': 1000.0},
 'accuracy': 0.9935,
 'macro avg': {'precision': 0.993583415597236,
  'recall': 0.9935,
  'f1-score': 0.9934997253633966,
  'support': 2000.0},
 'weighted avg': {'precision': 0.993583415597236,
  'recall': 0.9935,
  'f1-score': 0.9934997253633966,
  'support': 2000.0}}

In [55]:
challenge_preds = clf.predict(encoded_challenge_dataset)
classification_report(challenge_labels, challenge_preds, output_dict=True)

d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9995.0},
 '1': {'precision': 0.5001250312578145,
  'recall': 1.0,
  'f1-score': 0.6667777962993833,
  'support': 10000.0},
 'accuracy': 0.5001250312578145,
 'macro avg': {'precision': 0.25006251562890724,
  'recall': 0.5,
  'f1-score': 0.33338889814969164,
  'support': 19995.0},
 'weighted avg': {'precision': 0.2501250468906299,
  'recall': 0.5001250312578145,
  'f1-score': 0.3334722662162457,
  'support': 19995.0}}

# Encode Using Alphabet Arrays With Frequencies

In [56]:
def encode_with_freq(seq):
    seq = seq.upper()
    assert len(seq) == SEQ_LENGTH, "length of sequence must be {}".format(SEQ_LENGTH)
    encoding = [0]*len(all_chars)
    for i, char in enumerate(seq):
        encoding[all_chars.index(char)] += 1
    return encoding

encoded_with_freq_train_dataset = [encode_with_freq(seq) for seq in train_dataset]
encoded_with_freq_test_dataset = [encode_with_freq(seq) for seq in test_dataset]
encoded_with_freq_challenge_dataset = [encode_with_freq(seq) for seq in challenge_dataset]

In [57]:
clf_encoded_with_freq = MLPClassifier(hidden_layer_sizes=(100), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
clf_encoded_with_freq.fit(encoded_with_freq_train_dataset, train_labels)

Iteration 1, loss = 0.68582820
Validation score: 0.642500
Iteration 2, loss = 0.62864313
Validation score: 0.760000
Iteration 3, loss = 0.58608849
Validation score: 0.805000
Iteration 4, loss = 0.54995146
Validation score: 0.830000
Iteration 5, loss = 0.51820774
Validation score: 0.843750
Iteration 6, loss = 0.48938777
Validation score: 0.853750
Iteration 7, loss = 0.46366243
Validation score: 0.860000
Iteration 8, loss = 0.44009541
Validation score: 0.871250
Iteration 9, loss = 0.41659527
Validation score: 0.871250
Iteration 10, loss = 0.39573664
Validation score: 0.882500
Iteration 11, loss = 0.37645661
Validation score: 0.881250
Iteration 12, loss = 0.35831642
Validation score: 0.892500
Iteration 13, loss = 0.34229885
Validation score: 0.885000
Iteration 14, loss = 0.32618930
Validation score: 0.891250
Iteration 15, loss = 0.31167694
Validation score: 0.895000
Iteration 16, loss = 0.29825726
Validation score: 0.902500
Iteration 17, loss = 0.28720972
Validation score: 0.903750
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=100, max_iter=1000,
              random_state=42, verbose=True)

In [58]:
test_preds = clf_encoded_with_freq.predict(encoded_with_freq_test_dataset)
classification_report(test_labels, test_preds, output_dict=True)

{'0': {'precision': 0.965042372881356,
  'recall': 0.911,
  'f1-score': 0.9372427983539094,
  'support': 1000.0},
 '1': {'precision': 0.915719696969697,
  'recall': 0.967,
  'f1-score': 0.9406614785992218,
  'support': 1000.0},
 'accuracy': 0.939,
 'macro avg': {'precision': 0.9403810349255265,
  'recall': 0.9390000000000001,
  'f1-score': 0.9389521384765656,
  'support': 2000.0},
 'weighted avg': {'precision': 0.9403810349255265,
  'recall': 0.939,
  'f1-score': 0.9389521384765656,
  'support': 2000.0}}

In [59]:
challenge_preds = clf_encoded_with_freq.predict(encoded_with_freq_challenge_dataset)
classification_report(challenge_labels, challenge_preds, output_dict=True)

{'0': {'precision': 0.5,
  'recall': 0.03711855927963982,
  'f1-score': 0.06910682686038931,
  'support': 9995.0},
 '1': {'precision': 0.500129849893523,
  'recall': 0.9629,
  'f1-score': 0.6583256418145147,
  'support': 10000.0},
 'accuracy': 0.5001250312578145,
 'macro avg': {'precision': 0.5000649249467615,
  'recall': 0.5000092796398199,
  'f1-score': 0.363716234337452,
  'support': 19995.0},
 'weighted avg': {'precision': 0.500064941182057,
  'recall': 0.5001250312578145,
  'f1-score': 0.36378990510701364,
  'support': 19995.0}}

# Encode as ASCII indices shifted to 0

In [66]:
def encode_with_ascii_shifted_to_zero(seq):
    seq = seq.upper()
    assert len(seq) == SEQ_LENGTH, "length of sequence must be {}".format(SEQ_LENGTH)
    encoding = [0]*SEQ_LENGTH
    for i, char in enumerate(seq):
        encoding[i] = ord(char) - ord('A')
    return encoding

encoded_with_ascii_shifted_to_zero_train_dataset = [encode_with_ascii_shifted_to_zero(seq) for seq in train_dataset]
encoded_with_ascii_shifted_to_zero_test_dataset = [encode_with_ascii_shifted_to_zero(seq) for seq in test_dataset]
encoded_with_ascii_shifted_to_zero_challenge_dataset = [encode_with_ascii_shifted_to_zero(seq) for seq in challenge_dataset]

In [75]:
mlp_encoded_with_ascii_shifted_to_zero = MLPClassifier(hidden_layer_sizes=(9,9), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
mlp_encoded_with_ascii_shifted_to_zero.fit(encoded_with_ascii_shifted_to_zero_train_dataset, train_labels)

Iteration 1, loss = 1.32416578
Validation score: 0.528750
Iteration 2, loss = 0.95393623
Validation score: 0.522500
Iteration 3, loss = 0.80655650
Validation score: 0.538750
Iteration 4, loss = 0.72736210
Validation score: 0.580000
Iteration 5, loss = 0.68438535
Validation score: 0.601250
Iteration 6, loss = 0.65335498
Validation score: 0.631250
Iteration 7, loss = 0.61706697
Validation score: 0.675000
Iteration 8, loss = 0.54369544
Validation score: 0.778750
Iteration 9, loss = 0.44295066
Validation score: 0.841250
Iteration 10, loss = 0.34327966
Validation score: 0.896250
Iteration 11, loss = 0.25384204
Validation score: 0.928750
Iteration 12, loss = 0.20276966
Validation score: 0.940000
Iteration 13, loss = 0.17127767
Validation score: 0.957500
Iteration 14, loss = 0.14769964
Validation score: 0.958750
Iteration 15, loss = 0.12880724
Validation score: 0.967500
Iteration 16, loss = 0.11204375
Validation score: 0.970000
Iteration 17, loss = 0.09919714
Validation score: 0.976250
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(9, 9), max_iter=1000,
              random_state=42, verbose=True)

In [76]:
# Print MLP weights
mlp_encoded_with_ascii_shifted_to_zero.coefs_[0].shape, mlp_encoded_with_ascii_shifted_to_zero.coefs_[1].shape

((9, 9), (9, 9))

In [77]:
test_preds = mlp_encoded_with_ascii_shifted_to_zero.predict(encoded_with_ascii_shifted_to_zero_test_dataset)
classification_report(test_labels, test_preds, output_dict=True)

{'0': {'precision': 1.0,
  'recall': 0.993,
  'f1-score': 0.9964877069744105,
  'support': 1000.0},
 '1': {'precision': 0.9930486593843099,
  'recall': 1.0,
  'f1-score': 0.9965122072745392,
  'support': 1000.0},
 'accuracy': 0.9965,
 'macro avg': {'precision': 0.9965243296921549,
  'recall': 0.9964999999999999,
  'f1-score': 0.9964999571244748,
  'support': 2000.0},
 'weighted avg': {'precision': 0.9965243296921549,
  'recall': 0.9965,
  'f1-score': 0.9964999571244748,
  'support': 2000.0}}

In [78]:
challenge_preds = mlp_encoded_with_ascii_shifted_to_zero.predict(encoded_with_ascii_shifted_to_zero_challenge_dataset)
classification_report(challenge_labels, challenge_preds, output_dict=True)

{'0': {'precision': 1.0,
  'recall': 0.8367183591795898,
  'f1-score': 0.9111014271707157,
  'support': 9995.0},
 '1': {'precision': 0.859697386519945,
  'recall': 1.0,
  'f1-score': 0.9245562130177514,
  'support': 10000.0},
 'accuracy': 0.9183795948987247,
 'macro avg': {'precision': 0.9298486932599725,
  'recall': 0.918359179589795,
  'f1-score': 0.9178288200942335,
  'support': 19995.0},
 'weighted avg': {'precision': 0.9298311510477344,
  'recall': 0.9183795948987247,
  'f1-score': 0.9178305023630317,
  'support': 19995.0}}